In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')
import pickle
from tqdm import tqdm
import copy

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score

In [46]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False
print(cuda, device)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)

%cd drive/MyDrive/ime_project/
!ls

True cuda:0
[Errno 2] No such file or directory: 'drive/MyDrive/ime_project/'
/content/drive/MyDrive/ime_project
capston_EDA.ipynb  done		   EDA_1030.ipynb  feature_selection2.ipynb  test
checkpoint.pt	   EDA_1003.ipynb  EDA_1101.ipynb  feature_selection.ipynb   test_result
data		   EDA_1010.ipynb  EDA_1102.ipynb  lightning_logs	     trial.ipynb
DL		   EDA_1019.ipynb  EDA_1201.ipynb  result		     var.ipynb


# pre-task

In [47]:
with open('result/final_selection.pkl', 'rb') as file:
    final_selection = pickle.load(file)

features = []
for i in final_selection.keys():
    # print(i)
    for j in final_selection[i]:
        if j not in features:
            features.append(j)
            # print(features)
        else: pass

# features.append('cdate')

target = ['plant2_temperature','plant3_temperature','plant4_temperature','plant5_temperature','plant6_temperature','plant7_temperature','plant8_temperature','plant9_temperature','plant10_temperature']

In [48]:
# 앞선 feature selection으로 선택한 feature들로만 이루어진 training dataset
df_train = pd.read_csv("result/dataset.csv")#, index_col="Unnamed: 0")
df_train.rename(columns={'Unnamed: 0':'cdate'},inplace=True)

df_train["cdate"] = pd.to_datetime(df_train["cdate"])
min_date = df_train["cdate"].min()
df_train['time_idx'] = df_train["cdate"].apply(lambda x: (x - min_date).total_seconds()/60/10) # 10분단위로 timestamp
df_train['time_idx'] = df_train['time_idx'].astype(int)

df_train.set_index("cdate", inplace=True)
df_train = df_train[features]

df_train.columns = df_train.columns.str.lower()
df_train['onoff'] = df_train.loc[:, 'ahu1_inv':'ahu10_inv'].apply(lambda row: 1 if any(row != 0) else 0, axis=1)

df_train.drop("plant1_temperature", axis=1, inplace=True)

# 각 온도 열에 대해 결측값 표시 변수 추가
for col in df_train.columns:
    missing_col_name = col + "_missing"
    df_train[missing_col_name] = df_train[col].isna().astype(int)

    # 결측값을 평균값으로 채우기
    df_train[col].fillna(df_train[col].mean(), inplace=True)
    df_train.drop(missing_col_name, inplace=True, axis=1)

print(df_train.isna().any().any())
print(len(df_train.columns))
df_train = df_train.loc[:"2023-07-31 23:00:00"]
df_train.head()

False
27


,ahu1_inv,ahu2_inv,ahu3_inv,ahu4_inv,ahu5_inv,ahu7_inv,ahu8_inv,ahu9_inv,ahu10_inv,ahu11_inv,...,plant5_temperature,plant6_temperature,plant7_temperature,plant7_humidity,plant8_temperature,plant10_temperature,ahu13_inv,plant4_humidity,plant9_temperature,onoff
cdate,,,,,,,,,,,,,,,,,,,,,
2023-01-01 08:00:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,25.84,25.83,25.70,40.07,26.31,23.36,19.713945,39.42,23.09,1
2023-01-01 08:10:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,25.88,25.95,25.70,41.96,26.38,23.73,19.713945,39.62,23.49,1
2023-01-01 08:20:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,25.90,25.98,25.72,42.36,26.48,23.91,19.713945,39.86,23.60,1
2023-01-01 08:30:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,26.01,26.05,25.90,41.91,26.53,23.99,19.713945,40.07,23.76,1
2023-01-01 08:40:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,26.08,26.10,26.03,41.60,26.65,24.16,19.713945,40.13,23.94,1


In [49]:
lag3_feature = [item for item in df_train.columns if item not in target]
lag3_feature

['ahu1_inv',
 'ahu2_inv',
 'ahu3_inv',
 'ahu4_inv',
 'ahu5_inv',
 'ahu7_inv',
 'ahu8_inv',
 'ahu9_inv',
 'ahu10_inv',
 'ahu11_inv',
 'ahu12_inv',
 'ahu14_inv',
 'plant2_humidity',
 'plant3_humidity',
 'plant7_humidity',
 'ahu13_inv',
 'plant4_humidity',
 'onoff']

In [50]:
df_train[lag3_feature] = df_train[lag3_feature].shift(3)
df_train = df_train.bfill()

print(df_train.isna().any().any())

df_train.head()

False


,ahu1_inv,ahu2_inv,ahu3_inv,ahu4_inv,ahu5_inv,ahu7_inv,ahu8_inv,ahu9_inv,ahu10_inv,ahu11_inv,...,plant5_temperature,plant6_temperature,plant7_temperature,plant7_humidity,plant8_temperature,plant10_temperature,ahu13_inv,plant4_humidity,plant9_temperature,onoff
cdate,,,,,,,,,,,,,,,,,,,,,
2023-01-01 08:00:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,25.84,25.83,25.70,40.07,26.31,23.36,19.713945,39.42,23.09,1.0
2023-01-01 08:10:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,25.88,25.95,25.70,40.07,26.38,23.73,19.713945,39.42,23.49,1.0
2023-01-01 08:20:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,25.90,25.98,25.72,40.07,26.48,23.91,19.713945,39.42,23.60,1.0
2023-01-01 08:30:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,26.01,26.05,25.90,40.07,26.53,23.99,19.713945,39.42,23.76,1.0
2023-01-01 08:40:00,35.0,18.90246,35.0,19.266979,18.445291,18.043542,20.834108,19.026433,19.656569,19.619934,...,26.08,26.10,26.03,41.96,26.65,24.16,19.713945,39.62,23.94,1.0


In [51]:
y = df_train.loc[:,target]
X = df_train.drop(columns=target)

ms = MinMaxScaler()
ss = StandardScaler()

X_ss = ss.fit_transform(X)
y_ms = ms.fit_transform(y)

len_train = int(len(df_train)*0.8) # training 80% , validation 20%

X_train = X_ss[:len_train, :]
X_test = X_ss[len_train:, :]

y_train = y_ms[:len_train, :]
y_test = y_ms[len_train:, :]

print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape)

X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

window_size = 6

X_train_tensors_f = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], window_size, -1))
X_test_tensors_f = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], window_size, -1))

print("Training Shape", X_train_tensors_f.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_f.shape, y_test_tensors.shape)

Training Shape (24380, 18) (24380, 9)
Testing Shape (6095, 18) (6095, 9)
Training Shape torch.Size([24380, 6, 3]) torch.Size([24380, 9])
Testing Shape torch.Size([6095, 6, 3]) torch.Size([6095, 9])


In [52]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)
        self.fc_1 =  nn.Linear(hidden_size, 128)
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(x.device)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(x.device)

        output, (hn, cn) = self.lstm(x, (h_0, c_0))

        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.dropout(out)
        out = self.relu(out)
        # out = out[:,-1,:]
        out = self.fc(out)
        return out

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [53]:
num_epochs = 1000000
learning_rate = 1e-6

input_size = X_train_tensors_f.size(-1)

early_stopping = EarlyStopping(patience=10000, verbose=False) # 모델이 더 이상 성능이 향상되지 않을 때 학습을 조기에 중단하는 기법

hidden_size = 32
num_layers = 1
num_classes = y_train_tensors.shape[1]

model = LSTM(num_classes, input_size, hidden_size, num_layers, X_train_tensors_f.shape[1])
model.to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 1e-8) # weight decay는 모델의 복잡성을 줄이고 과적합 방지하는 정규화 기법
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1) # schedular는 학습 과정 중 learning rate 조정하여 wegiht 조정하여 성능 개선하는 기

In [54]:
# torch.save(model.state_dict(), 'result/lstm_6win_3timestep_selection_parameters.pth')
model.load_state_dict(torch.load('result/lstm_6win_3timestep_selection_parameters.pth'))

<All keys matched successfully>

# test_1002

In [73]:
df_test = pd.read_csv('test/2023-10-02.csv', index_col='cdate')
# print(df_test.columns)
# features.remove('plant1_temperature')
features_low = [i.lower() for i in features]
features_low.remove('plant1_temperature')
df_test = df_test[features_low]

df_test.columns = df_test.columns.str.lower()
df_test['onoff'] = df_test.loc[:, 'ahu1_inv':'ahu10_inv'].apply(lambda row: 1 if any(row != 0) else 0, axis=1)

# 각 온도 열에 대해 결측값 표시 변수 추가
for col in df_test.columns:
    missing_col_name = col + "_missing"
    df_test[missing_col_name] = df_test[col].isna().astype(int)

    # 결측값을 평균값으로 채우기
    df_test[col].fillna(df_test[col].mean(), inplace=True)
    df_test.drop(missing_col_name, inplace=True, axis=1)

print(df_test.isna().any().any())
print(len(df_test.columns))

### time-step=3
df_test[lag3_feature] = df_test[lag3_feature].shift(3)
df_test = df_test.bfill()
# df_test = df_test.loc
new_target = [i for i in target if i not in ["plant9_temperature","plant10_temperature"]]
test_df2 = pd.concat([df_test[new_target ][:'2023-10-02 09:50:00'],pd.read_csv("test_result/test_1002.csv", index_col='Unnamed: 0')['2023-10-02 10:00:00':]])
test_df2
print(test_df2.index)

False
27
Index(['2023-10-02 06:00:00', '2023-10-02 06:10:00', '2023-10-02 06:20:00',
       '2023-10-02 06:30:00', '2023-10-02 06:40:00', '2023-10-02 06:50:00',
       '2023-10-02 07:00:00', '2023-10-02 07:10:00', '2023-10-02 07:20:00',
       '2023-10-02 07:30:00', '2023-10-02 07:40:00', '2023-10-02 07:50:00',
       '2023-10-02 08:00:00', '2023-10-02 08:10:00', '2023-10-02 08:20:00',
       '2023-10-02 08:30:00', '2023-10-02 08:40:00', '2023-10-02 08:50:00',
       '2023-10-02 09:00:00', '2023-10-02 09:10:00', '2023-10-02 09:20:00',
       '2023-10-02 09:30:00', '2023-10-02 09:40:00', '2023-10-02 09:50:00',
       '2023-10-02 10:00:00', '2023-10-02 10:10:00', '2023-10-02 10:20:00',
       '2023-10-02 10:30:00', '2023-10-02 10:40:00', '2023-10-02 10:50:00',
       '2023-10-02 11:00:00', '2023-10-02 11:10:00', '2023-10-02 11:20:00',
       '2023-10-02 11:30:00', '2023-10-02 11:40:00', '2023-10-02 11:50:00',
       '2023-10-02 12:00:00', '2023-10-02 12:10:00', '2023-10-02 12:20:00',
   

# 1004

In [74]:
df_test = pd.read_csv('test/2023-10-04.csv', index_col='cdate')
# print(df_test.columns)
# features.remove('plant1_temperature')
features_low = [i.lower() for i in features]
features_low.remove('plant1_temperature')
df_test = df_test[features_low]

df_test.columns = df_test.columns.str.lower()
df_test['onoff'] = df_test.loc[:, 'ahu1_inv':'ahu10_inv'].apply(lambda row: 1 if any(row != 0) else 0, axis=1)

# 각 온도 열에 대해 결측값 표시 변수 추가
for col in df_test.columns:
    missing_col_name = col + "_missing"
    df_test[missing_col_name] = df_test[col].isna().astype(int)

    # 결측값을 평균값으로 채우기
    df_test[col].fillna(df_test[col].mean(), inplace=True)
    df_test.drop(missing_col_name, inplace=True, axis=1)

print(df_test.isna().any().any())
print(len(df_test.columns))

### time-step=3
df_test[lag3_feature] = df_test[lag3_feature].shift(3)
df_test = df_test.bfill()
# df_test = df_test.loc[:'2023-10-02 09:50:00']

df_con = pd.concat([df_test[new_target].loc[:'2023-10-04 09:50:00'],pd.read_csv("test_result/test_1004.csv", index_col='Unnamed: 0').loc['2023-10-04 10:00:00':]])
print(df_con.index)
df_test4 = pd.concat([test_df2, df_con])
df_test4

False
27
Index(['2023-10-04 06:00:00', '2023-10-04 06:10:00', '2023-10-04 06:20:00',
       '2023-10-04 06:30:00', '2023-10-04 06:40:00', '2023-10-04 06:50:00',
       '2023-10-04 07:00:00', '2023-10-04 07:10:00', '2023-10-04 07:20:00',
       '2023-10-04 07:30:00', '2023-10-04 07:40:00', '2023-10-04 07:50:00',
       '2023-10-04 08:00:00', '2023-10-04 08:10:00', '2023-10-04 08:20:00',
       '2023-10-04 08:30:00', '2023-10-04 08:40:00', '2023-10-04 08:50:00',
       '2023-10-04 09:00:00', '2023-10-04 09:10:00', '2023-10-04 09:20:00',
       '2023-10-04 09:30:00', '2023-10-04 09:40:00', '2023-10-04 09:50:00',
       '2023-10-04 10:00:00', '2023-10-04 10:10:00', '2023-10-04 10:20:00',
       '2023-10-04 10:30:00', '2023-10-04 10:40:00', '2023-10-04 10:50:00',
       '2023-10-04 11:00:00', '2023-10-04 11:10:00', '2023-10-04 11:20:00',
       '2023-10-04 11:30:00', '2023-10-04 11:40:00', '2023-10-04 11:50:00',
       '2023-10-04 12:00:00', '2023-10-04 12:10:00', '2023-10-04 12:20:00',
   

,plant2_temperature,plant3_temperature,plant4_temperature,plant5_temperature,plant6_temperature,plant7_temperature,plant8_temperature
2023-10-02 06:00:00,28.160000,28.490000,28.150000,28.110000,27.970000,27.950000,29.030000
2023-10-02 06:10:00,28.150000,28.500000,28.140000,28.110000,27.970000,27.980000,29.030000
2023-10-02 06:20:00,28.130000,28.510000,28.150000,28.100000,28.000000,27.960000,29.040000
2023-10-02 06:30:00,28.140000,28.500000,28.170000,28.150000,28.000000,27.980000,29.020000
2023-10-02 06:40:00,28.130000,28.520000,28.190000,28.180000,28.020000,28.000000,29.060000
...,...,...,...,...,...,...,...
2023-10-04 21:20:00,27.342886,27.172375,27.073160,27.076374,27.008800,27.793270,28.905390
2023-10-04 21:30:00,28.234404,28.146967,28.119516,28.023990,27.696003,28.685274,31.231665
2023-10-04 21:40:00,29.392440,29.364243,29.330574,29.214039,28.787120,29.917732,32.565773
2023-10-04 21:50:00,29.313961,29.068678,29.021751,28.870800,28.570547,29.930082,30.452827


# 1006

In [75]:
df_test = pd.read_csv('test/2023-10-06.csv', index_col='cdate')
# print(df_test.columns)
# features.remove('plant1_temperature')
features_low = [i.lower() for i in features]
features_low.remove('plant1_temperature')
df_test = df_test[features_low]

df_test.columns = df_test.columns.str.lower()
df_test['onoff'] = df_test.loc[:, 'ahu1_inv':'ahu10_inv'].apply(lambda row: 1 if any(row != 0) else 0, axis=1)

# 각 온도 열에 대해 결측값 표시 변수 추가
for col in df_test.columns:
    missing_col_name = col + "_missing"
    df_test[missing_col_name] = df_test[col].isna().astype(int)

    # 결측값을 평균값으로 채우기
    df_test[col].fillna(df_test[col].mean(), inplace=True)
    df_test.drop(missing_col_name, inplace=True, axis=1)

print(df_test.isna().any().any())
print(len(df_test.columns))

### time-step=3
df_test[lag3_feature] = df_test[lag3_feature].shift(3)
df_test = df_test.bfill()
# df_test = df_test.loc[:'2023-10-02 09:50:00']

df_con = pd.concat([df_test[new_target][:'2023-10-06 09:50:00'],pd.read_csv("test_result/test_1006.csv", index_col='Unnamed: 0')['2023-10-06 10:00:00':]])
df_test6 = pd.concat([df_test4, df_con])
df_test6
print(df_con.index)

False
27
Index(['2023-10-06 06:00:00', '2023-10-06 06:10:00', '2023-10-06 06:20:00',
       '2023-10-06 06:30:00', '2023-10-06 06:40:00', '2023-10-06 06:50:00',
       '2023-10-06 07:00:00', '2023-10-06 07:10:00', '2023-10-06 07:20:00',
       '2023-10-06 07:30:00', '2023-10-06 07:40:00', '2023-10-06 07:50:00',
       '2023-10-06 08:00:00', '2023-10-06 08:10:00', '2023-10-06 08:20:00',
       '2023-10-06 08:30:00', '2023-10-06 08:40:00', '2023-10-06 08:50:00',
       '2023-10-06 09:00:00', '2023-10-06 09:10:00', '2023-10-06 09:20:00',
       '2023-10-06 09:30:00', '2023-10-06 09:40:00', '2023-10-06 09:50:00',
       '2023-10-06 10:00:00', '2023-10-06 10:10:00', '2023-10-06 10:20:00',
       '2023-10-06 10:30:00', '2023-10-06 10:40:00', '2023-10-06 10:50:00',
       '2023-10-06 11:00:00', '2023-10-06 11:10:00', '2023-10-06 11:20:00',
       '2023-10-06 11:30:00', '2023-10-06 11:40:00', '2023-10-06 11:50:00',
       '2023-10-06 12:00:00', '2023-10-06 12:10:00', '2023-10-06 12:20:00',
   

# 1008

In [76]:
df_test = pd.read_csv('test/2023-10-08.csv', index_col='cdate')
# print(df_test.columns)
# features.remove('plant1_temperature')
features_low = [i.lower() for i in features]
features_low.remove('plant1_temperature')
df_test = df_test[features_low]

df_test.columns = df_test.columns.str.lower()
df_test['onoff'] = df_test.loc[:, 'ahu1_inv':'ahu10_inv'].apply(lambda row: 1 if any(row != 0) else 0, axis=1)

# 각 온도 열에 대해 결측값 표시 변수 추가
for col in df_test.columns:
    missing_col_name = col + "_missing"
    df_test[missing_col_name] = df_test[col].isna().astype(int)

    # 결측값을 평균값으로 채우기
    df_test[col].fillna(df_test[col].mean(), inplace=True)
    df_test.drop(missing_col_name, inplace=True, axis=1)

print(df_test.isna().any().any())
print(len(df_test.columns))

### time-step=3
df_test[lag3_feature] = df_test[lag3_feature].shift(3)
df_test = df_test.bfill()
# df_test = df_test.loc[:'2023-10-02 09:50:00']

df_con = pd.concat([df_test[new_target][:'2023-10-08 09:50:00'],pd.read_csv("test_result/test_1008.csv", index_col='Unnamed: 0')['2023-10-08 10:00:00':]])
print(df_con.index)
df_test8 = pd.concat([df_test6, df_con])
df_test8

False
27
Index(['2023-10-08 06:00:00', '2023-10-08 06:10:00', '2023-10-08 06:20:00',
       '2023-10-08 06:30:00', '2023-10-08 06:40:00', '2023-10-08 06:50:00',
       '2023-10-08 07:00:00', '2023-10-08 07:10:00', '2023-10-08 07:20:00',
       '2023-10-08 07:30:00', '2023-10-08 07:40:00', '2023-10-08 07:50:00',
       '2023-10-08 08:00:00', '2023-10-08 08:10:00', '2023-10-08 08:20:00',
       '2023-10-08 08:30:00', '2023-10-08 08:40:00', '2023-10-08 08:50:00',
       '2023-10-08 09:00:00', '2023-10-08 09:10:00', '2023-10-08 09:20:00',
       '2023-10-08 09:30:00', '2023-10-08 09:40:00', '2023-10-08 09:50:00',
       '2023-10-08 10:00:00', '2023-10-08 10:10:00', '2023-10-08 10:20:00',
       '2023-10-08 10:30:00', '2023-10-08 10:40:00', '2023-10-08 10:50:00',
       '2023-10-08 11:00:00', '2023-10-08 11:10:00', '2023-10-08 11:20:00',
       '2023-10-08 11:30:00', '2023-10-08 11:40:00', '2023-10-08 11:50:00',
       '2023-10-08 12:00:00', '2023-10-08 12:10:00', '2023-10-08 12:20:00',
   

,plant2_temperature,plant3_temperature,plant4_temperature,plant5_temperature,plant6_temperature,plant7_temperature,plant8_temperature
2023-10-02 06:00:00,28.160000,28.490000,28.150000,28.110000,27.970000,27.950000,29.030000
2023-10-02 06:10:00,28.150000,28.500000,28.140000,28.110000,27.970000,27.980000,29.030000
2023-10-02 06:20:00,28.130000,28.510000,28.150000,28.100000,28.000000,27.960000,29.040000
2023-10-02 06:30:00,28.140000,28.500000,28.170000,28.150000,28.000000,27.980000,29.020000
2023-10-02 06:40:00,28.130000,28.520000,28.190000,28.180000,28.020000,28.000000,29.060000
...,...,...,...,...,...,...,...
2023-10-08 21:20:00,27.408484,27.269130,27.043920,27.252136,26.977710,27.845076,29.237803
2023-10-08 21:30:00,28.340403,28.197716,28.095165,28.172050,27.691746,28.842043,31.782166
2023-10-08 21:40:00,28.244322,28.176947,28.187447,28.138681,27.720985,28.801378,32.331364
2023-10-08 21:50:00,28.298100,28.141390,28.021692,28.012392,27.927172,28.831556,30.197046


# 1010

In [84]:
df_test = pd.read_csv('test/2023-10-10.csv', index_col='cdate')
# print(df_test.columns)
# features.remove('plant1_temperature')
features_low = [i.lower() for i in features]
features_low.remove('plant1_temperature')
df_test = df_test[features_low]

df_test.columns = df_test.columns.str.lower()
df_test['onoff'] = df_test.loc[:, 'ahu1_inv':'ahu10_inv'].apply(lambda row: 1 if any(row != 0) else 0, axis=1)

# 각 온도 열에 대해 결측값 표시 변수 추가
for col in df_test.columns:
    missing_col_name = col + "_missing"
    df_test[missing_col_name] = df_test[col].isna().astype(int)

    # 결측값을 평균값으로 채우기
    df_test[col].fillna(df_test[col].mean(), inplace=True)
    df_test.drop(missing_col_name, inplace=True, axis=1)

print(df_test.isna().any().any())
print(len(df_test.columns))

### time-step=3
df_test[lag3_feature] = df_test[lag3_feature].shift(3)
df_test = df_test.bfill()
# df_test = df_test.loc[:'2023-10-02 09:50:00']

df_con = pd.concat([df_test[new_target][:'2023-10-10 09:50:00'],pd.read_csv("test_result/test_1010.csv", index_col='Unnamed: 0')['2023-10-10 10:00:00':]])
print(df_con.index)
df_test10 = pd.concat([df_test8, df_con])
df_test10

False
27
Index(['2023-10-10 06:00:00', '2023-10-10 06:10:00', '2023-10-10 06:20:00',
       '2023-10-10 06:30:00', '2023-10-10 06:40:00', '2023-10-10 06:50:00',
       '2023-10-10 07:00:00', '2023-10-10 07:10:00', '2023-10-10 07:20:00',
       '2023-10-10 07:30:00', '2023-10-10 07:40:00', '2023-10-10 07:50:00',
       '2023-10-10 08:00:00', '2023-10-10 08:10:00', '2023-10-10 08:20:00',
       '2023-10-10 08:30:00', '2023-10-10 08:40:00', '2023-10-10 08:50:00',
       '2023-10-10 09:00:00', '2023-10-10 09:10:00', '2023-10-10 09:20:00',
       '2023-10-10 09:30:00', '2023-10-10 09:40:00', '2023-10-10 09:50:00',
       '2023-10-10 10:00:00', '2023-10-10 10:10:00', '2023-10-10 10:20:00',
       '2023-10-10 10:30:00', '2023-10-10 10:40:00', '2023-10-10 10:50:00',
       '2023-10-10 11:00:00', '2023-10-10 11:10:00', '2023-10-10 11:20:00',
       '2023-10-10 11:30:00', '2023-10-10 11:40:00', '2023-10-10 11:50:00',
       '2023-10-10 12:00:00', '2023-10-10 12:10:00', '2023-10-10 12:20:00',
   

,plant2_temperature,plant3_temperature,plant4_temperature,plant5_temperature,plant6_temperature,plant7_temperature,plant8_temperature
2023-10-02 06:00:00,28.160000,28.490000,28.150000,28.110000,27.970000,27.950000,29.030000
2023-10-02 06:10:00,28.150000,28.500000,28.140000,28.110000,27.970000,27.980000,29.030000
2023-10-02 06:20:00,28.130000,28.510000,28.150000,28.100000,28.000000,27.960000,29.040000
2023-10-02 06:30:00,28.140000,28.500000,28.170000,28.150000,28.000000,27.980000,29.020000
2023-10-02 06:40:00,28.130000,28.520000,28.190000,28.180000,28.020000,28.000000,29.060000
...,...,...,...,...,...,...,...
2023-10-10 21:20:00,28.217684,28.087029,27.862154,27.961657,28.119411,28.842995,33.305940
2023-10-10 21:30:00,28.807238,28.659311,28.489975,28.605877,28.606028,29.442757,33.876064
2023-10-10 21:40:00,28.290663,28.112463,27.902748,28.034294,28.066193,28.983149,33.553930
2023-10-10 21:50:00,27.851484,27.800312,27.568245,27.785522,27.694600,28.588774,33.577770


In [85]:
df_test10.to_csv("test_result/final_test.csv")